In [1]:
import pandas as pd
from pandas import datetime
import numpy as np
import math
import re
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
data1 = pd.read_csv("C:/Users/shrini/Downloads/INFY.csv")

In [3]:
infy= data1[["Date","Close"]]

In [4]:
infy.head()

,Date,Close
0,2018-04-03,8.840
1,2018-04-04,8.915
2,2018-04-05,8.935
3,2018-04-06,8.745
4,2018-04-09,8.630


In [5]:
def add_datepart1(df, fldname, drop=True, time=False, errors="raise"):	

    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

def is_date(x): return np.issubdtype(x.dtype, np.datetime64)

In [6]:
infy["Date"] = pd.to_datetime(infy["Date"])
add_datepart1(infy,'Date')
infy.head()

C:\Users\shrini\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\shrini\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Close,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,8.840,2018,4,14,3,1,93,False,False,False,False,False,False,1522713600
1,8.915,2018,4,14,4,2,94,False,False,False,False,False,False,1522800000
2,8.935,2018,4,14,5,3,95,False,False,False,False,False,False,1522886400
3,8.745,2018,4,14,6,4,96,False,False,False,False,False,False,1522972800
4,8.630,2018,4,15,9,0,99,False,False,False,False,False,False,1523232000


In [7]:
infy=infy.dropna()
infy.head()

,Close,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,8.840,2018,4,14,3,1,93,False,False,False,False,False,False,1522713600
1,8.915,2018,4,14,4,2,94,False,False,False,False,False,False,1522800000
2,8.935,2018,4,14,5,3,95,False,False,False,False,False,False,1522886400
3,8.745,2018,4,14,6,4,96,False,False,False,False,False,False,1522972800
4,8.630,2018,4,15,9,0,99,False,False,False,False,False,False,1523232000


In [8]:
def train_test_split(a, n): return a[:n], a[n:]

n_valu = 50
n_traini = len(infy)-n_valu
X_Train, X_valu = train_test_split(infy.drop(columns=["Close"]), n_traini)
y_Train, y_valu = train_test_split(infy["Close"], n_traini)

X_Train.shape, y_Train.shape, X_valu.shape, y_valu.shape

((202, 13), (202,), (50, 13), (50,))

In [9]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators': [3,10,30], 'max_features': [2,4,6,8]},
    {'bootstrap' : [False],'n_estimators': [3,10], 'max_features': [2, 3, 4]}
]

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_Train, y_Train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [10]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 3}

In [11]:
cv_scores = grid_search.cv_results_
for mean_score, params in zip(cv_scores["mean_test_score"], cv_scores["params"]):
  print(np.sqrt(-mean_score), params)

0.44205957366249443 {'max_features': 2, 'n_estimators': 3}
0.42444942351098985 {'max_features': 2, 'n_estimators': 10}
0.44356358260702805 {'max_features': 2, 'n_estimators': 30}
0.5237140860186879 {'max_features': 4, 'n_estimators': 3}
0.4283526685610422 {'max_features': 4, 'n_estimators': 10}
0.43764533355366425 {'max_features': 4, 'n_estimators': 30}
0.382407710847639 {'max_features': 6, 'n_estimators': 3}
0.3892764693048571 {'max_features': 6, 'n_estimators': 10}
0.4200618113207073 {'max_features': 6, 'n_estimators': 30}
0.3652390826003509 {'max_features': 8, 'n_estimators': 3}
0.41954106674218217 {'max_features': 8, 'n_estimators': 10}
0.3916269458827803 {'max_features': 8, 'n_estimators': 30}
0.43763217396801307 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
0.39472075603839823 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
0.4281564016693304 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
0.4117682182904107 {'bootstrap': False, 'max_features':

In [13]:
from sklearn.linear_model import LinearRegression
reg=LinearRegression().fit(X_Train,y_Train)
reg.score(X_Train,y_Train)

0.2552882424284256

In [14]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(X_Train,y_Train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [15]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'alpha': [0.01,0.10,1.0], 'max_iter': [200,400,600,800]},
    {'normalize' : [True],'alpha': [0.01,0.10,1.0], 'max_iter': [200,400,600,800]}
]

grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_Train, y_Train)

C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to in

GridSearchCV(cv=5, error_score='raise',
       estimator=Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'alpha': [0.01, 0.1, 1.0], 'max_iter': [200, 400, 600, 800]}, {'normalize': [True], 'alpha': [0.01, 0.1, 1.0], 'max_iter': [200, 400, 600, 800]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [16]:
grid_search.best_params_

{'alpha': 0.1, 'max_iter': 200, 'normalize': True}

In [17]:
cv_scores = grid_search.cv_results_
for mean_score, params in zip(cv_scores["mean_test_score"], cv_scores["params"]):
  print(np.sqrt(-mean_score), params)

0.8733444662115513 {'alpha': 0.01, 'max_iter': 200}
0.8592497360556552 {'alpha': 0.01, 'max_iter': 400}
0.8446074285723306 {'alpha': 0.01, 'max_iter': 600}
0.8317498515002767 {'alpha': 0.01, 'max_iter': 800}
0.7965360756740075 {'alpha': 0.1, 'max_iter': 200}
0.8035717757756893 {'alpha': 0.1, 'max_iter': 400}
0.8035717757756893 {'alpha': 0.1, 'max_iter': 600}
0.8035717757756893 {'alpha': 0.1, 'max_iter': 800}
0.799409733577015 {'alpha': 1.0, 'max_iter': 200}
0.799409733577015 {'alpha': 1.0, 'max_iter': 400}
0.799409733577015 {'alpha': 1.0, 'max_iter': 600}
0.799409733577015 {'alpha': 1.0, 'max_iter': 800}
0.7775960853675794 {'alpha': 0.01, 'max_iter': 200, 'normalize': True}
0.7781753920073072 {'alpha': 0.01, 'max_iter': 400, 'normalize': True}
0.7781924153211927 {'alpha': 0.01, 'max_iter': 600, 'normalize': True}
0.7781924153211927 {'alpha': 0.01, 'max_iter': 800, 'normalize': True}
0.6930092892506027 {'alpha': 0.1, 'max_iter': 200, 'normalize': True}
0.6930092892506027 {'alpha': 0.1, 

In [18]:
from sklearn.linear_model import Ridge
model_lasso = Ridge()

In [20]:
clf1 = linear_model.Ridge(alpha=0.1)
clf1.fit(X_Train,y_Train)

C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.136957e-18
  overwrite_a=True).T


Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [21]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'alpha': [0.01,0.10,1.0], 'max_iter': [200,400,600,800]},
    {'normalize' : [True],'alpha': [0.01,0.10,1.0], 'max_iter': [200,400,600,800]}
]

grid_search = GridSearchCV(clf1, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_Train, y_Train)

C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.799293e-18
  overwrite_a=True).T
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.043355e-18
  overwrite_a=True).T
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.209712e-19
  overwrite_a=True).T
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.044735e-18
  overwrite_a=True).T
C:\U

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'alpha': [0.01, 0.1, 1.0], 'max_iter': [200, 400, 600, 800]}, {'normalize': [True], 'alpha': [0.01, 0.1, 1.0], 'max_iter': [200, 400, 600, 800]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [22]:
grid_search.best_params_

{'alpha': 1.0, 'max_iter': 400, 'normalize': True}

In [23]:
cv_scores = grid_search.cv_results_
for mean_score, params in zip(cv_scores["mean_test_score"], cv_scores["params"]):
  print(np.sqrt(-mean_score), params)

9.398239320995993 {'alpha': 0.01, 'max_iter': 200}
9.398239314502877 {'alpha': 0.01, 'max_iter': 400}
9.398239320995993 {'alpha': 0.01, 'max_iter': 600}
9.398239320995993 {'alpha': 0.01, 'max_iter': 800}
1.998762808272583 {'alpha': 0.1, 'max_iter': 200}
1.9987628101160435 {'alpha': 0.1, 'max_iter': 400}
1.9987628088695615 {'alpha': 0.1, 'max_iter': 600}
1.998762808272583 {'alpha': 0.1, 'max_iter': 800}
0.8054646099486573 {'alpha': 1.0, 'max_iter': 200}
0.8054646099486567 {'alpha': 1.0, 'max_iter': 400}
0.8054646099403424 {'alpha': 1.0, 'max_iter': 600}
0.8054646099486573 {'alpha': 1.0, 'max_iter': 800}
0.8458041849793334 {'alpha': 0.01, 'max_iter': 200, 'normalize': True}
0.8458041849792722 {'alpha': 0.01, 'max_iter': 400, 'normalize': True}
0.8458041849792803 {'alpha': 0.01, 'max_iter': 600, 'normalize': True}
0.8458041849793334 {'alpha': 0.01, 'max_iter': 800, 'normalize': True}
0.8369096058763711 {'alpha': 0.1, 'max_iter': 200, 'normalize': True}
0.8369096058763571 {'alpha': 0.1, 'm